# Credit Ratings on Earnings Call Dates

Use earnings call data (symbol and date for companies) and credit ratings data at the level of issuances to create a dataset of credit ratings on earnings call dates.

In [ ]:
# packages
import pandas as pd
import pandasql as ps

In [ ]:
# Load credit ratings data
# C:\Users\ijyli\Box\STAT 222 Capstone\Intermediate Data\combined_credit_rating_data.csv
credit_ratings = pd.read_csv('~/Box/STAT 222 Capstone/Intermediate Data/combined_credit_rating_data.csv')
# Rename Date to rating_date, Next Rating Date or End of Data to next_rating_date_or_end_of_data, Symbol to ticker
credit_ratings = credit_ratings.rename(columns={'Date': 'rating_date', 'Next Rating Date or End of Data': 'next_rating_date_or_end_of_data', 'Symbol': 'ticker'})
# Limit rating agency and scope
credit_ratings = credit_ratings[credit_ratings['Rating Agency Name'].str.contains('Standard & Poor\'s') & (credit_ratings['Year'] >= 2010) & (credit_ratings['Year'] <= 2016)]
credit_ratings

In [ ]:
# Load selected variables from earnings call data
earnings_call_dates = pd.read_csv('earnings_call_dates.csv')
# Rename variable to earnings_call_date
earnings_call_dates

## Join together

In [ ]:
query = '''
SELECT a.*, b.earnings_call_date
FROM credit_ratings a
LEFT JOIN earnings_call_dates b
ON a.ticker = b.ticker
    AND a.rating_date <= b.earnings_call_date
        AND b.earnings_call_date <= a.next_rating_date_or_end_of_data
'''

# Run query
credit_ratings_on_earnings_call_dates = ps.sqldf(query, locals())

In [ ]:
# Check for uniqueness by ticker and earnings_call_date
credit_ratings_on_earnings_call_dates.groupby(['ticker', 'earnings_call_date']).size().reset_index(name='count').sort_values(by='count', ascending=False)

## Credit Rating One Call Ahead, Two Calls Ahead, etc.

In [ ]:
# Sort by ticker and earnings_call_date
credit_ratings_on_earnings_call_dates = credit_ratings_on_earnings_call_dates.sort_values(by=['ticker', 'earnings_call_date'])

# Get next earnings call date and next value of rating
credit_ratings_on_earnings_call_dates['next_earnings_call_date'] = credit_ratings_on_earnings_call_dates.groupby('ticker')['earnings_call_date'].shift(-1)
credit_ratings_on_earnings_call_dates['rating_on_next_earnings_call_date'] = credit_ratings_on_earnings_call_dates.groupby('ticker')['Rating'].shift(-1)

# Two earnings call dates later
# lead_2_earnings_call_date
credit_ratings_on_earnings_call_dates['lead_2_earnings_call_date'] = credit_ratings_on_earnings_call_dates.groupby('ticker')['earnings_call_date'].shift(-2)
# lead_2_rating
credit_ratings_on_earnings_call_dates['lead_2_rating'] = credit_ratings_on_earnings_call_dates.groupby('ticker')['Rating'].shift(-2)

## Output CSV

In [ ]:
# Save to csv
credit_ratings_on_earnings_call_dates.to_csv('~/Box/STAT 222 Capstone/Intermediate Data/credit_ratings_on_earnings_call_dates.csv', index=False)